In [ ]:
# [START import]
from ortools.constraint_solver import routing_enums_pb2
from ortools.constraint_solver import pywrapcp
import pandas as pd
import numpy as np

In [ ]:
# Import Distance Matrix 
df_distance = pd.read_excel('df_distance_matrix.xlsx', index_col=0)

# Transform to Numpy Array
distance_matrix = df_distance.to_numpy()


In [ ]:
distance_matrix

In [62]:
from haversine import haversine, Unit


def create_distance_matrix(coordinates):
    num_locations = len(coordinates)
    distance_matrix = np.zeros((num_locations, num_locations))

    for i in range(num_locations):
        for j in range(num_locations):
            distance_matrix[i][j] = round(haversine(
                coordinates[i], coordinates[j], unit=Unit.KILOMETERS))

    print(distance_matrix)

    return distance_matrix

            
    





# Example usage:
# file_name = 'route-optimization\df_distance_matrix.xlsx'
coordinates = [
    (40.7128, -74.0060),  # New York City
    (34.0522, -118.2437),  # Los Angeles
    (41.8781, -87.6298),   # Chicago
    (29.7604, -95.3698),   # Houston
    (33.4484, -112.0740),  # Phoenix
    (29.9511, -90.0715),   # New Orleans
    (39.9526, -75.1652),   # Philadelphia
    (32.7767, -96.7970),   # Dallas
    (37.7749, -122.4194),  # San Francisco
    (38.9072, -77.0369),   # Washington D.C.
    (35.2271, -80.8431),   # Charlotte
    (42.3601, -71.0589),   # Boston
    (33.7490, -84.3880),   # Atlanta
    (47.6062, -122.3321),  # Seattle
    (25.7617, -80.1918),   # Miami
    (39.2904, -76.6122)    # Baltimore
]

create_distance_matrix(coordinates)

[[   0.         3935.75169089 1144.29285451 2281.34797079 3443.7024156
  1880.30221207  129.61295056 2205.73026413 4129.09186839  327.58316611
   854.59948037  306.10891703 1200.31845645 3865.53368969 1757.96365327
   272.55242091]
 [3935.75169089    0.         2803.97537999 2206.26707578  574.26661957
  2686.65910772 3843.4587094  1991.99034926  559.12134935 3692.00646227
  3403.78348301 4169.61296312 3110.33752464 1545.30778985 3758.77464822
  3725.23316128]
 [1144.29285451 2803.97537999    0.         1515.8072005  2336.34421504
  1344.1606562  1068.00411385 1294.95706631 2984.91178672  955.18502221
   945.48518534 1365.64137057  947.50795018 2788.8611148  1917.31254043
   973.05953815]
 [2281.34797079 2206.26707578 1515.8072005     0.         1632.16913597
   511.34869703 2156.27384557  361.77646429 2643.04950367 1961.09473241
  1489.86408341 2581.33519002 1128.26521426 3040.5044903  1556.79664946
  2012.65898183]
 [3443.7024156   574.26661957 2336.34421504 1632.16913597    0.
  211

array([[   0.        , 3935.75169089, 1144.29285451, 2281.34797079,
        3443.7024156 , 1880.30221207,  129.61295056, 2205.73026413,
        4129.09186839,  327.58316611,  854.59948037,  306.10891703,
        1200.31845645, 3865.53368969, 1757.96365327,  272.55242091],
       [3935.75169089,    0.        , 2803.97537999, 2206.26707578,
         574.26661957, 2686.65910772, 3843.4587094 , 1991.99034926,
         559.12134935, 3692.00646227, 3403.78348301, 4169.61296312,
        3110.33752464, 1545.30778985, 3758.77464822, 3725.23316128],
       [1144.29285451, 2803.97537999,    0.        , 1515.8072005 ,
        2336.34421504, 1344.1606562 , 1068.00411385, 1294.95706631,
        2984.91178672,  955.18502221,  945.48518534, 1365.64137057,
         947.50795018, 2788.8611148 , 1917.31254043,  973.05953815],
       [2281.34797079, 2206.26707578, 1515.8072005 ,    0.        ,
        1632.16913597,  511.34869703, 2156.27384557,  361.77646429,
        2643.04950367, 1961.09473241, 1489.86

In [60]:
dis

NameError: name 'distance_matrix_1' is not defined

In [ ]:
# Create dictionnary with data
data = {}
data['distance_matrix'] = distance_matrix
print("{:,} destinations".format(len(data['distance_matrix'][0]) - 1))

# Orders quantity (Boxes)
data['demands'] = [0, 1, 1, 2, 4, 2, 4, 8, 8, 1, 2, 1, 2, 4, 4, 8, 8]
# Vehicles Capacities (Boxes)
data['vehicle_capacities'] = [15, 15, 15, 15]
# Fleet informations
# Number of vehicles
data['num_vehicles'] = 4
# Location of the depot
data['depot'] = 0

In [ ]:
# Orders quantity (Boxes)
data['demands'] = [0, 1, 1, 2, 4, 2, 4, 8, 6, 1, 2, 1, 2, 75, 4, 8, 8]
# Vehicles Capacities (Boxes)
data['vehicle_capacities'] = [15, 15, 105, 15]
# Fleet informations
# Number of vehicles
data['num_vehicles'] = 4
# Location of the depot
data['depot'] = 0

In [ ]:
def distance_callback(from_index, to_index):
    """Returns the distance between the two nodes."""
    # Convert from routing variable Index to distance matrix NodeIndex.
    from_node = manager.IndexToNode(from_index)
    to_node = manager.IndexToNode(to_index)
    return data['distance_matrix'][from_node][to_node]

In [ ]:
def demand_callback(from_index):
    """Returns the demand of the node."""
    # Convert from routing variable Index to demands NodeIndex.
    from_node = manager.IndexToNode(from_index)
    return data['demands'][from_node]

In [ ]:
# Create the routing index manager.
manager = pywrapcp.RoutingIndexManager(len(data['distance_matrix']),
                                       data['num_vehicles'], data['depot'])

# Create Routing Model
routing = pywrapcp.RoutingModel(manager)

# Create and register a transit callback.
transit_callback_index = routing.RegisterTransitCallback(distance_callback)

# Define cost of each arc.
routing.SetArcCostEvaluatorOfAllVehicles(transit_callback_index)

# Add Capacity constraint.
demand_callback_index = routing.RegisterUnaryTransitCallback(
    demand_callback)
routing.AddDimensionWithVehicleCapacity(demand_callback_index,
    0,  # null capacity slack
    data['vehicle_capacities'],  # vehicle maximum capacities
    True,  # start cumul to zero
    'Capacity')

# Setting first solution heuristic.
search_parameters = pywrapcp.DefaultRoutingSearchParameters()
search_parameters.first_solution_strategy = (
    routing_enums_pb2.FirstSolutionStrategy.PATH_CHEAPEST_ARC)
search_parameters.local_search_metaheuristic = (
    routing_enums_pb2.LocalSearchMetaheuristic.GUIDED_LOCAL_SEARCH)
search_parameters.time_limit.FromSeconds(1)

# Solve the problem.
solution = routing.SolveWithParameters(search_parameters)

In [ ]:
if solution:
    total_distance = 0
    total_load = 0
    for vehicle_id in range(data['num_vehicles']):
        index = routing.Start(vehicle_id)
        plan_output = 'Route for driver {}:\n'.format(vehicle_id)
        route_distance = 0
        route_load = 0
        while not routing.IsEnd(index):
            node_index = manager.IndexToNode(index)
            route_load += data['demands'][node_index]
            plan_output += ' {0} Parcels({1}) -> '.format(node_index, route_load)
            previous_index = index
            index = solution.Value(routing.NextVar(index))
            route_distance += routing.GetArcCostForVehicle(
                previous_index, index, vehicle_id)
        plan_output += ' {0} Parcels({1})\n'.format(manager.IndexToNode(index),
                                                 route_load)
        plan_output += 'Distance of the route: {} (m)\n'.format(route_distance)
        plan_output += 'Parcels Delivered: {} (parcels)\n'.format(route_load)
        print(plan_output)
        total_distance += route_distance
        total_load += route_load
    print('Total distance of all routes: {:,} (m)'.format(total_distance))
    print('Parcels Delivered: {:,}/{:,}'.format(total_load, sum(data['demands'])))
else:
    print('No Solution')